In [1]:
import duckdb 

In [7]:
#Load the httpfs to load data from a remote repo. Run this command only once. 
conn = duckdb.connect('taxi_NYC4.db')
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

In [6]:
conn.execute("CREATE TABLE green_taxi AS SELECT * FROM read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet');")

Error: An error occurred while trying to automatically install the required extension 'httpfs':
Failed to download extension "httpfs" at URL "http://extensions.duckdb.org/v1.0.0/windows_amd64/httpfs.duckdb_extension.gz"
Extension "httpfs" is an existing extension.
 (ERROR Read)